In [2]:
from tools import *
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
from scipy.stats import multivariate_normal
from scipy.linalg import fractional_matrix_power
from scipy.stats import wishart
import sys
import time

In [3]:
def reflect(x,xL,xU):
    n=0; side=0; e=0
    if (x<xL):
        e = xL-x; side=0 
    elif(x>xU):
        e = x-xU; side=1
    n = int(e/(xU-xL));
    if(n%2):
        side = 1-side
    e -= n*(xU-xL)
    x = xU-e if side else xL+e
    return(x)

In [4]:
def logtargetpdf(x, target, kernel):
    lnp = 0.0
    if (target == 'N01'):
        lnp = -x*x/2
    elif (target == 'TwoNormal'):
        lnp = np.log(0.25 * norm_pdf(x, loc=-1, scale=(1/2)) + 0.75 * norm_pdf(x, loc=1, scale=(1/2)))
    elif (target == 'TwoT4'):
        m = 0.75
        st = np.sqrt(37.0)/8.0
        lnp = np.log(0.75 * PDFt4(x,-m,st) + 0.25 * PDFt4(x, m, st))
    elif (target == 'Gamma'):
        xL = 0
        a = 4; b = 2
        lnp = x + (-b*np.exp(x) + (a-1)*x) if ('Mirror' in kernel) else ((-b*x + (a-1)*np.log(x)) if (x >= xL) else -500)
    elif (target == 'Uniform'):
        xL = -np.sqrt(3); xU = np.sqrt(3)
        if ('Mirror' in kernel):
            lnp = x + np.log(xU-xL) - 2*np.log(np.exp(x)+1)
        else:
            lnp = 0 if ((x >= xL) & (x<= xU)) else -500
    return (lnp)

In [12]:
#MCMC sampling implementation
np.random.seed(121)
begin_total = time.time()
nround = int(10e5)
burnin = int(10e4)
size = len(range(-burnin,nround))
sigmaset = [0.1, 0.3, 0.5, 0.7, 0.9, 1, 1.1, 1.3, 1.5, 1.7,
                    1.8, 1.9, 2, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.8,
                    3.0, 3.2, 3.5, 4, 4.5, 5, 6, 7, 8]
kernelset = ['Uniform','Gaussian','Bactrian','Box','Airplane','StrawHat','MirrorU','MirrorN']
targetset = ['N01','TwoNormal','TwoT4','Gamma','Uniform']

sample_Normal = np.random.normal(size = size)
sample_Uniform = np.random.uniform(low=-np.sqrt(3), high=np.sqrt(3), size=size)
sample_bactrian = random_bactrian(size = size)
sample_box = random_box(size=size)
sample_airplane = random_airplane(size=size)
sample_strawhat = random_strawhat(size=size)
target_result = {}

for target in targetset:
    eff_kernel = {}; Pjump_kernel = {}; Exx2_kernel = {}; Rho_kernel={}; Sample_kernel={}
    for kernel in kernelset:
        Eff = []; Pjump = []; Exx2=[]; Rho=[]; Sample=[]
        if ('Mirror' in kernel) & ((target=='Gamma')| (target=='Uniform')):
            mu_star = 0.405 if target == 'Gamma' else 0.116
        else:
            mu_star = 0.1
        for sigma in sigmaset:
            begin_loop = time.time()
            exx2 = 0; accept = 0; 
            x = abs(np.random.random() - 0.5)
            y = np.zeros(nround)
            for j in range(-burnin,nround):
                if (j==0):
                    accept = 0;
                if kernel == 'Gaussian':
                    xnew = x + sample_Normal[j+burnin] * sigma
                elif kernel == 'Uniform':
                    xnew = x + sample_Uniform[j+burnin] *sigma
                elif kernel == 'Bactrian':
                    xnew = x + sample_bactrian[j+burnin] *sigma
                elif kernel == 'Box':
                    xnew = x + sample_box[j+burnin] *sigma
                elif kernel == 'Airplane':
                    xnew = x + sample_airplane[j+burnin] *sigma
                elif kernel == 'StrawHat':
                    xnew = x + sample_strawhat[j+burnin] *sigma
                elif kernel == 'MirrorU':
                    xnew = (2 * mu_star - x) + sample_Uniform[j+burnin] * sigma
                elif kernel == 'MirrorN':
                    xnew = (2 * mu_star - x) + sample_Normal[j+burnin] * sigma

                if (target == 'Gamma') & (xnew<0) & ('Mirror' not in kernel):
                    xnew = -xnew
                elif ((target == 'Uniform') & (not (-np.sqrt(3) <= xnew <= np.sqrt(3))) & ('Mirror' not in kernel)):
                    xnew = reflect(xnew, xL=-np.sqrt(3), xU=np.sqrt(3))

                pi_dif = logtargetpdf(xnew,target,kernel) - logtargetpdf(x,target,kernel)
                if ((pi_dif>0) | (np.exp(pi_dif)>np.random.random())):
                    x = xnew
                    accept += 1
                if (j>=0):
                    if ((target=='Gamma') & ('Mirror' in kernel)):
                        y[j] = np.exp(x)
                    elif ((target=='Uniform') & ('Mirror' in kernel)):
                        y[j] = (np.sqrt(3)*np.exp(x)-np.sqrt(3))/(np.exp(x)+1)
                    else:
                        y[j] = x
            eff = (Eff_IntegratedCorrelationTime(y))
            rho = [rho_k(y,i) for i in range(1,11)]
            exx2 = sum(np.square(y[1:] - y[:-1]))
            Eff.append(eff)
            Pjump.append(accept/nround)
            Exx2.append(exx2/nround)
            Rho.append(rho)
            Sample.append(y.copy())
            print ('Target:',target,'|Kernel:',kernel,'|sigma:',sigma,'|eff:',round(eff,4), '|Pjump:', round(accept/nround,4), '|Exx2:', round(exx2/nround,4),
                   '|rho1:', round(rho[0],4), '|rho2:', round(rho[1],4) ,'|Time:',round((time.time()-begin_loop),2),sep='')
        eff_kernel[kernel] = Eff
        Pjump_kernel[kernel] = Pjump
        Exx2_kernel[kernel] = Exx2
        Rho_kernel[kernel] = Rho
        Sample_kernel[kernel] = Sample
    target_result[target] = [eff_kernel,Pjump_kernel,Exx2_kernel,Rho_kernel,Sample_kernel]
print('总共用时: %f'% (time.time()-begin_total))

Target:N01|Kernel:Uniform|sigma:0.1|eff:0.0025|Pjump:0.9656|Exx2:0.0095|rho1:0.9952|rho2:0.9905|Time:13.81
Target:N01|Kernel:Uniform|sigma:0.3|eff:0.0191|Pjump:0.8969|Exx2:0.076|rho1:0.962|rho2:0.9255|Time:11.49
Target:N01|Kernel:Uniform|sigma:0.5|eff:0.0476|Pjump:0.8296|Exx2:0.1863|rho1:0.9063|rho2:0.8221|Time:11.37
Target:N01|Kernel:Uniform|sigma:0.7|eff:0.0822|Pjump:0.7648|Exx2:0.3186|rho1:0.84|rho2:0.7086|Time:10.67
Target:N01|Kernel:Uniform|sigma:0.9|eff:0.1193|Pjump:0.7038|Exx2:0.4558|rho1:0.7718|rho2:0.6016|Time:10.73
Target:N01|Kernel:Uniform|sigma:1|eff:0.1414|Pjump:0.6748|Exx2:0.521|rho1:0.739|rho2:0.5532|Time:11.23
Target:N01|Kernel:Uniform|sigma:1.1|eff:0.159|Pjump:0.6463|Exx2:0.5822|rho1:0.7082|rho2:0.5106|Time:10.46
Target:N01|Kernel:Uniform|sigma:1.3|eff:0.193|Pjump:0.5936|Exx2:0.6921|rho1:0.6559|rho2:0.44|Time:10.57
Target:N01|Kernel:Uniform|sigma:1.5|eff:0.2251|Pjump:0.5439|Exx2:0.7745|rho1:0.6137|rho2:0.3856|Time:10.68
Target:N01|Kernel:Uniform|sigma:1.7|eff:0.2472|Pj

Target:N01|Kernel:Bactrian|sigma:2.8|eff:0.3408|Pjump:0.2223|Exx2:1.0915|rho1:0.4537|rho2:0.2135|Time:10.71
Target:N01|Kernel:Bactrian|sigma:3.0|eff:0.3027|Pjump:0.1975|Exx2:1.061|rho1:0.4711|rho2:0.2339|Time:10.71
Target:N01|Kernel:Bactrian|sigma:3.2|eff:0.2728|Pjump:0.1741|Exx2:1.0105|rho1:0.4943|rho2:0.2596|Time:11.54
Target:N01|Kernel:Bactrian|sigma:3.5|eff:0.2153|Pjump:0.1444|Exx2:0.9226|rho1:0.5392|rho2:0.3121|Time:11.41
Target:N01|Kernel:Bactrian|sigma:4|eff:0.1461|Pjump:0.1064|Exx2:0.7685|rho1:0.6146|rho2:0.4053|Time:11.01
Target:N01|Kernel:Bactrian|sigma:4.5|eff:0.1015|Pjump:0.08|Exx2:0.6267|rho1:0.6872|rho2:0.4989|Time:10.95
Target:N01|Kernel:Bactrian|sigma:5|eff:0.0754|Pjump:0.0612|Exx2:0.5081|rho1:0.7467|rho2:0.5808|Time:11.61
Target:N01|Kernel:Bactrian|sigma:6|eff:0.0402|Pjump:0.0369|Exx2:0.3191|rho1:0.8408|rho2:0.7207|Time:11.81
Target:N01|Kernel:Bactrian|sigma:7|eff:0.0259|Pjump:0.0242|Exx2:0.2108|rho1:0.8951|rho2:0.8088|Time:12.21
Target:N01|Kernel:Bactrian|sigma:8|eff:

Target:N01|Kernel:StrawHat|sigma:1.8|eff:0.3485|Pjump:0.3968|Exx2:1.0917|rho1:0.4546|rho2:0.2277|Time:11.3
Target:N01|Kernel:StrawHat|sigma:1.9|eff:0.361|Pjump:0.372|Exx2:1.1237|rho1:0.4383|rho2:0.2109|Time:10.86
Target:N01|Kernel:StrawHat|sigma:2|eff:0.3725|Pjump:0.3493|Exx2:1.1497|rho1:0.4254|rho2:0.1989|Time:11.19
Target:N01|Kernel:StrawHat|sigma:2.1|eff:0.3811|Pjump:0.3271|Exx2:1.1688|rho1:0.416|rho2:0.1893|Time:11.09
Target:N01|Kernel:StrawHat|sigma:2.2|eff:0.3943|Pjump:0.3062|Exx2:1.1785|rho1:0.4106|rho2:0.1829|Time:10.74
Target:N01|Kernel:StrawHat|sigma:2.3|eff:0.3914|Pjump:0.2869|Exx2:1.1822|rho1:0.4084|rho2:0.1805|Time:11.04
Target:N01|Kernel:StrawHat|sigma:2.4|eff:0.3814|Pjump:0.2692|Exx2:1.1889|rho1:0.4081|rho2:0.1821|Time:10.8
Target:N01|Kernel:StrawHat|sigma:2.5|eff:0.3751|Pjump:0.2505|Exx2:1.1744|rho1:0.4138|rho2:0.1849|Time:11.22
Target:N01|Kernel:StrawHat|sigma:2.6|eff:0.3688|Pjump:0.2349|Exx2:1.1695|rho1:0.4177|rho2:0.1899|Time:10.94
Target:N01|Kernel:StrawHat|sigma:2.

Target:TwoNormal|Kernel:Uniform|sigma:0.3|eff:0.0104|Pjump:0.8225|Exx2:0.0662|rho1:0.9672|rho2:0.9377|Time:30.05
Target:TwoNormal|Kernel:Uniform|sigma:0.5|eff:0.0267|Pjump:0.7199|Exx2:0.1479|rho1:0.9265|rho2:0.8664|Time:29.98
Target:TwoNormal|Kernel:Uniform|sigma:0.7|eff:0.0519|Pjump:0.6364|Exx2:0.2377|rho1:0.8811|rho2:0.7858|Time:29.31
Target:TwoNormal|Kernel:Uniform|sigma:0.9|eff:0.0872|Pjump:0.5773|Exx2:0.3408|rho1:0.8307|rho2:0.6953|Time:28.72
Target:TwoNormal|Kernel:Uniform|sigma:1|eff:0.106|Pjump:0.5511|Exx2:0.3924|rho1:0.8032|rho2:0.6489|Time:28.54
Target:TwoNormal|Kernel:Uniform|sigma:1.1|eff:0.1237|Pjump:0.5284|Exx2:0.4469|rho1:0.7765|rho2:0.6047|Time:27.9
Target:TwoNormal|Kernel:Uniform|sigma:1.3|eff:0.158|Pjump:0.4886|Exx2:0.5574|rho1:0.7213|rho2:0.5225|Time:27.57
Target:TwoNormal|Kernel:Uniform|sigma:1.5|eff:0.1925|Pjump:0.4544|Exx2:0.6658|rho1:0.668|rho2:0.4493|Time:27.67
Target:TwoNormal|Kernel:Uniform|sigma:1.7|eff:0.2203|Pjump:0.4214|Exx2:0.7424|rho1:0.6296|rho2:0.4007|

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


...

Target:Gamma|Kernel:Box|sigma:2.5|eff:0.3389|Pjump:0.4405|Exx2:1.1885|rho1:0.4042|rho2:0.2265|Time:13.0
Target:Gamma|Kernel:Box|sigma:2.6|eff:0.3457|Pjump:0.4385|Exx2:1.21|rho1:0.3978|rho2:0.225|Time:12.97
Target:Gamma|Kernel:Box|sigma:2.8|eff:0.3628|Pjump:0.4349|Exx2:1.2582|rho1:0.3713|rho2:0.2063|Time:13.25
Target:Gamma|Kernel:Box|sigma:3.0|eff:0.3729|Pjump:0.428|Exx2:1.2842|rho1:0.3575|rho2:0.1944|Time:13.02
Target:Gamma|Kernel:Box|sigma:3.2|eff:0.3851|Pjump:0.4187|Exx2:1.2992|rho1:0.3541|rho2:0.1869|Time:12.91
Target:Gamma|Kernel:Box|sigma:3.5|eff:0.3776|Pjump:0.3994|Exx2:1.2784|rho1:0.3601|rho2:0.1833|Time:12.84
Target:Gamma|Kernel:Box|sigma:4|eff:0.3765|Pjump:0.3627|Exx2:1.2172|rho1:0.3886|rho2:0.1908|Time:12.52
Target:Gamma|Kernel:Box|sigma:4.5|eff:0.3294|Pjump:0.3227|Exx2:1.1101|rho1:0.4454|rho2:0.2292|Time:13.27
Target:Gamma|Kernel:Box|sigma:5|eff:0.2813|Pjump:0.2834|Exx2:0.9916|rho1:0.5036|rho2:0.2803|Time:12.67
Target:Gamma|Kernel:Box|sigma:6|eff:0.1858|Pjump:0.2105|Exx2:0.7

Target:Gamma|Kernel:MirrorU|sigma:1.1|eff:0.3655|Pjump:0.4157|Exx2:1.1488|rho1:0.4268|rho2:0.1972|Time:15.31
Target:Gamma|Kernel:MirrorU|sigma:1.3|eff:0.3012|Pjump:0.3609|Exx2:0.986|rho1:0.5092|rho2:0.2732|Time:15.08
Target:Gamma|Kernel:MirrorU|sigma:1.5|eff:0.2472|Pjump:0.3154|Exx2:0.8597|rho1:0.5711|rho2:0.3403|Time:15.51
Target:Gamma|Kernel:MirrorU|sigma:1.7|eff:0.2166|Pjump:0.2799|Exx2:0.7664|rho1:0.619|rho2:0.3932|Time:15.1
Target:Gamma|Kernel:MirrorU|sigma:1.8|eff:0.2002|Pjump:0.2637|Exx2:0.7078|rho1:0.6446|rho2:0.4242|Time:15.27
Target:Gamma|Kernel:MirrorU|sigma:1.9|eff:0.1842|Pjump:0.2503|Exx2:0.6791|rho1:0.6593|rho2:0.4445|Time:15.2
Target:Gamma|Kernel:MirrorU|sigma:2|eff:0.177|Pjump:0.2386|Exx2:0.6426|rho1:0.6782|rho2:0.4671|Time:14.98
Target:Gamma|Kernel:MirrorU|sigma:2.1|eff:0.1692|Pjump:0.2273|Exx2:0.6155|rho1:0.6928|rho2:0.4865|Time:15.33
Target:Gamma|Kernel:MirrorU|sigma:2.2|eff:0.1557|Pjump:0.2175|Exx2:0.5923|rho1:0.706|rho2:0.5054|Time:15.08
Target:Gamma|Kernel:MirrorU

Target:Uniform|Kernel:Uniform|sigma:4.5|eff:0.8238|Pjump:1.0|Exx2:1.8025|rho1:0.099|rho2:0.0113|Time:17.08
Target:Uniform|Kernel:Uniform|sigma:5|eff:0.779|Pjump:1.0|Exx2:1.7533|rho1:0.1243|rho2:0.0173|Time:17.09
Target:Uniform|Kernel:Uniform|sigma:6|eff:1.0068|Pjump:1.0|Exx2:1.9962|rho1:0.0003|rho2:0.0013|Time:17.3
Target:Uniform|Kernel:Uniform|sigma:7|eff:1.1827|Pjump:1.0|Exx2:2.1757|rho1:-0.0873|rho2:0.0085|Time:17.51
Target:Uniform|Kernel:Uniform|sigma:8|eff:0.9994|Pjump:1.0|Exx2:1.9953|rho1:0.001|rho2:-0.0003|Time:17.44
Target:Uniform|Kernel:Gaussian|sigma:0.1|eff:0.0022|Pjump:1.0|Exx2:0.0097|rho1:0.9952|rho2:0.9905|Time:17.82
Target:Uniform|Kernel:Gaussian|sigma:0.3|eff:0.0183|Pjump:1.0|Exx2:0.0816|rho1:0.9592|rho2:0.9218|Time:15.8
Target:Uniform|Kernel:Gaussian|sigma:0.5|eff:0.0505|Pjump:1.0|Exx2:0.2112|rho1:0.8944|rho2:0.8045|Time:15.47
Target:Uniform|Kernel:Gaussian|sigma:0.7|eff:0.0998|Pjump:1.0|Exx2:0.384|rho1:0.808|rho2:0.6591|Time:15.47
Target:Uniform|Kernel:Gaussian|sigma:

Target:Uniform|Kernel:Box|sigma:2.2|eff:1.8044|Pjump:1.0|Exx2:2.5883|rho1:-0.2942|rho2:0.0907|Time:16.48
Target:Uniform|Kernel:Box|sigma:2.3|eff:2.0864|Pjump:1.0|Exx2:2.717|rho1:-0.3584|rho2:0.133|Time:16.75
Target:Uniform|Kernel:Box|sigma:2.4|eff:2.4046|Pjump:1.0|Exx2:2.8317|rho1:-0.4171|rho2:0.1779|Time:16.55
Target:Uniform|Kernel:Box|sigma:2.5|eff:2.743|Pjump:1.0|Exx2:2.9445|rho1:-0.4715|rho2:0.2258|Time:16.77
Target:Uniform|Kernel:Box|sigma:2.6|eff:3.1119|Pjump:1.0|Exx2:3.0399|rho1:-0.5194|rho2:0.2735|Time:16.77
Target:Uniform|Kernel:Box|sigma:2.8|eff:3.9297|Pjump:1.0|Exx2:3.1882|rho1:-0.5958|rho2:0.3596|Time:17.03
Target:Uniform|Kernel:Box|sigma:3.0|eff:4.76|Pjump:1.0|Exx2:3.286|rho1:-0.6463|rho2:0.4237|Time:16.95
Target:Uniform|Kernel:Box|sigma:3.2|eff:5.1877|Pjump:1.0|Exx2:3.3347|rho1:-0.6704|rho2:0.4573|Time:17.13
Target:Uniform|Kernel:Box|sigma:3.5|eff:4.9243|Pjump:1.0|Exx2:3.3169|rho1:-0.6599|rho2:0.4445|Time:17.41
Target:Uniform|Kernel:Box|sigma:4|eff:3.3179|Pjump:1.0|Exx2:3

Target:Uniform|Kernel:MirrorU|sigma:0.9|eff:2.2971|Pjump:0.8062|Exx2:2.8777|rho1:-0.4322|rho2:0.196|Time:27.72
Target:Uniform|Kernel:MirrorU|sigma:1|eff:2.0948|Pjump:0.7868|Exx2:2.7625|rho1:-0.3806|rho2:0.1532|Time:27.42
Target:Uniform|Kernel:MirrorU|sigma:1.1|eff:1.8383|Pjump:0.7672|Exx2:2.6591|rho1:-0.3275|rho2:0.1162|Time:27.94
Target:Uniform|Kernel:MirrorU|sigma:1.3|eff:1.523|Pjump:0.73|Exx2:2.4686|rho1:-0.2294|rho2:0.0647|Time:27.75
Target:Uniform|Kernel:MirrorU|sigma:1.5|eff:1.2289|Pjump:0.6942|Exx2:2.272|rho1:-0.1376|rho2:0.0372|Time:27.75
Target:Uniform|Kernel:MirrorU|sigma:1.7|eff:1.0462|Pjump:0.6603|Exx2:2.1036|rho1:-0.0524|rho2:0.0225|Time:27.42
Target:Uniform|Kernel:MirrorU|sigma:1.8|eff:0.9601|Pjump:0.6439|Exx2:2.0289|rho1:-0.0145|rho2:0.0231|Time:27.66
Target:Uniform|Kernel:MirrorU|sigma:1.9|eff:0.8893|Pjump:0.6277|Exx2:1.9553|rho1:0.0213|rho2:0.0276|Time:28.05
Target:Uniform|Kernel:MirrorU|sigma:2|eff:0.8214|Pjump:0.6122|Exx2:1.8805|rho1:0.0586|rho2:0.0324|Time:27.34
Tar

In [13]:
np.save('mcmc_target_result',target_result)